In [10]:
import os
import json
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import pandas as pd

import processing_pdf
from model_summarizer.ModelSummarizer import SummarizationModel, load_data
from gpt_summary import get_summaries
from generate_ppt import *

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

pd.options.display.max_colwidth = 1000

Variables here:

In [21]:
filename = "1901.00039v2"
summary_models = ["allenai/led-large-16384-arxiv", "gpt-3.5-turbo-0125"]
model_name = summary_models[0]
use_gpt_ppt_parsing = True

# 1. Processing PDF

In [12]:
pdf_file = f"{filename}.pdf"
project_path = os.getcwd()
project_data_path = project_path + "/data"
project_processed_data_path = project_path + "/processed"
if not os.path.exists(project_data_path):
    os.makedirs(project_data_path)
if not os.path.exists(project_processed_data_path):
    os.makedirs(project_processed_data_path)

In [13]:
doc, total_text, total_pages = processing_pdf.open_file(project_data_path + "/" + pdf_file)

# Use processing_pdf.auto_find_toc(), which will clean up the original toc
# table_of_content = doc.get_toc()
table_of_content = processing_pdf.auto_find_toc(doc)
display(table_of_content)

Auto generated table of content:


[[1, 'I Introduction', 1],
 [1, 'II Related work', 2],
 [1, 'III Our proposed method', 2],
 [2, 'III-A Density map estimation', 2],
 [2, 'III-B Method overview', 3],
 [2, 'III-C Backbone sub-network', 3],
 [2, 'III-D Mask prediction branch', 3],
 [2, 'III-E Mask-aware density density regressor', 4],
 [2, 'III-F Implementation details', 5],
 [1, 'IV Experiment', 5],
 [2, 'IV-A Evaluation metrics ', 5],
 [2, 'IV-B Datasets', 5],
 [2, 'IV-C Analysis of the proposed approaches', 6],
 [2, 'IV-D Ablation study', 6],
 [2, 'IV-E Comparison with the state-of-the-art', 8],
 [1, 'V Conclusion', 8],
 [1, 'References', 9]]

In [14]:
#uncomment this list to customize table-of-content
# table_of_content = [[1, 'I. INTRODUCTION', 1],
#  [1, 'II. SFC BASED ON IPV6 SEGMENT ROUTING', 2],
#  [1, 'III. DESIGN OF THE SRV6 PROXY', 4],
#  [2, 'A. General Concepts and State-of-the-art', 4],
#  [2, 'B. SRNKv1', 5],
#  [2, 'C. SRNKv2', 7],
#  [2, 'D. Implementation of other SR proxy types', 8],
#  [1, 'IV. TESTING ENVIRONMENT', 8],
#  [1, 'V. PERFORMANCE ANALYSIS', 9],
#  [1, 'VII. CONCLUSIONS', 11]]

# separate content into sections
processing_pdf.clear_processed_folder(project_processed_data_path)
title, authors, other_info, abstract = processing_pdf.find_meta_data(doc, table_of_content)
df_meta = pd.DataFrame([title, abstract]).T
df_meta.columns = ["Title", "Abstract"]
ds, json_dict = processing_pdf.separate_content(total_text, table_of_content)
processing_pdf.save_dataframe(ds, df_meta, json_dict, project_processed_data_path,  pdf_file.rsplit(".", 1)[0])
# extract images
processing_pdf.find_images(doc, table_of_content, total_pages, project_processed_data_path)

# display(ds)
print(f"\nTitle: {title}")
print(f"\nAuthors:{authors}")
print(f"\nOther info: {other_info}")
print(f"\nAbstract:{abstract}")


starting looking for all the sections according to the provided section title info...
save the dataframe to c:\Academics\MSBA Semester 2\CS5242\Project\PDF_conversion/processed/1901.00039v2.csv
save the dataframe to c:\Academics\MSBA Semester 2\CS5242\Project\PDF_conversion/processed/1901.00039v2_meta.csv
save the dataframe to c:\Academics\MSBA Semester 2\CS5242\Project\PDF_conversion/processed/1901.00039v2.json

Title: arXiv:1901.00039v2  [cs.CV]  20 Jun 2019

Authors:

Other info: 

Abstract:


In [15]:
# open json file
with open(f"{project_processed_data_path}/{filename}.json") as f:
    data = json.load(f)
    for item in data:
        display(item)

{'Section_Num': 'Abstract',
 'Section': 'Abstract',
 'Text': 'ieee transactions on circuits and systems for video technology 1 mask-aware networks for crowd counting shengqin jiang, xiaobo lu, yinjie lei, lingqiao liu abstract {em dash}crowd counting problem aims to count the number of objects within an image or a frame in the videos and is usually solved by estimating the density map generated from the object location annotations. the values in the density map, by nature, take two possible states: zero indicating no object around, a non-zero value indicating the existence of objects and the value denoting the local object density. in contrast to traditional methods which do not differentiate the density prediction of these two states, we propose to use a dedicated network branch to predict the object/non-object mask and then combine its prediction with the input image to produce the density map. our rationale is that the mask prediction could be better modeled as a binary segmentation

{'Section_Num': 'I',
 'Section': 'I Introduction',
 'Text': 'c rowd counting is a significant topic for crowd understanding and analysis -, which has attracted much attention in multimedia and computer vision community due to large and practical demands for better management, safety and security , . it aims to count the number of objects within an image or a frame in the videos and is a very challenging problem because the objects-of-interest, e.g, people, can occur at a variety of scales, with heavy occlusions and cluttered visual appearances. also, due to the difficulty in providing highly detailed annotations such as object bounding boxes or instance-level segmentation masks, existing datasets usually adopt a weak-level annotating scheme by labeling each object with a dot inside. these challenges make the traditional detection based approach less robust - and most existing methods - choose to solve this problem by estimating a density map generated from the dot-level annotation. onc

{'Section_Num': 'II',
 'Section': 'II Related work',
 'Text': 'to date, many approaches have been proposed to study the issues existing in crowd counting , -. here we present a brief review of the related work. for a more detailed survey of crowd counting, we refer the readers to , , . detection seems to be a straightforward solution for crowd counting. the most early methods use hand-crafted features such as haar wavelets , histogram oriented gradients to model the pedestrian, which are then fed to classifiers to distinguish whether there is pedestrian or not. initially, , studied the monocular pedestrian detection by a diverse set of low-level feature-based systems. although monocularbased methods work well in a low density region, the performance is severely affected when they meet the crowded scenes with occlusion and scene clutter. to further consider this issue, more information of the pedestrian is taken into account. zhao et al. used multiple partially occluded human hypotheses

{'Section_Num': 'III',
 'Section': 'III Our proposed method',
 'Text': "a. density map estimation before elaborating the design of our network, we first briefly introduce the creation of the ground-truth density maps and the related training losses. this paper considers the case that a point-wise annotation is provided for training images. specifically, a dot is annotated within each object-of-interest, i.e, people head. this pointwise annotation is further converted into a density map: d = x xibelong to a g, ieee transactions on circuits and systems for video technology 3 fig. 1. an overview of our proposed method. it contains three modules: the backbone, the mask prediction branch and the mask-aware density regression branch. where x belong to r2denotes the image coordinate and xi denotes the annotated head location. g denotes a gaussian kernel with xi as the mean vector and {greek small letter sigma}2 as the empirically chosen variance term. a typical choice of {greek small letter s

{'Section_Num': 'IV',
 'Section': 'IV Experiment',
 'Text': "in this section, we conduct experiments on three challenging public datasets to demonstrate the effectiveness our proposed method including shanghaitech dataset , ucf cc 50 dataset and worldexpo' 10 dataset . the purposes of our experiments are threefold: verify if the proposed mask-aware strategies lead to significant improvement over the baselines. identify the most effective mask-aware density estimation solution. compare our proposed approach against the state-of-the-art methods. in our experiments, we use shanghaitech dataset a to achieve the first and the second objective. the identified best-performed solution will then be compared against the state-of-the-art on all three datasets. in what follows, we present the evaluation criterion and datasets in our experiments. then we present a detailed analysis of the proposed solutions and identify the most effective one. finally, we compare our method with other state-of-thea

{'Section_Num': 'V',
 'Section': 'V Conclusion',
 'Text': "in this paper, we address the crowd counting problem with deep neural networks. our main discovery is the benefit ieee transactions on circuits and systems for video technology 9 table iii the performance comparison on the shanghaitech dataset. part a part b method mae mse mae mse cc-counting 181.8 277.7 32.0 49.8 fcn 126.5 173.5 23.8 33.1 mcnn 110.2 173.2 26.4 41.3 tdf-net 97.5 145.1 20.7 32.8 switching-net 90.4 135.0 21.6 33.4 ba-cnn 20.2 35.6 netvlad 107.6 169.3 21.4 33.9 cp-cnn 73.6 106.4 20.1 30.1 csrnet 68.2 115.0 10.6 16.0 our method 65.7 107.8 11.7 16.4 our model csr 61.8 100.0 8.6 13.3 table iv the performance comparison on the ucf cc 50 dataset. method mae mse cc-counting 467.0 498.5 fcn 338.6 424.5 mcnn 377.6 509.1 tdf-net 354.7 491.4 switching-net 318.1 439.2 ba-cnn 409.5 563.7 netvlad 311.3 401.8 cp-cnn 295.8 320.9 csrnet 266.1 397.5 our method 283.3 411.6 our model csr 245.4 349.3 table v the performance compariso

{'Section_Num': 'References',
 'Section': 'References',
 'Text': '',
 'Subsections': [],
 'Groundtruth': ''}

# 2. Model Summarisation

In [16]:
if model_name == "gpt-3.5-turbo-0125":
    with open(f"processed/{filename}.json", encoding='utf-8') as f:
        data = json.load(f)
    get_summaries(data)
    with open(f"gpt_summaries/{filename}.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
else:
    summarizer = SummarizationModel(model_name)
    model = summarizer.model

    # path for the extracted pdf's json file
    data_file_path = os.path.join('processed', f'{filename}.json')
    # Load  the json file for summarization
    pdf_data = load_data(data_file_path)

    summarizer_model = SummarizationModel(model_name)

    #Write the final summary to the summary jsonfile
    output_file =os.path.join('model_summarizer/results/model-summary_results.json')
    summarizer.summarize_pdf(pdf_data, output_file, summarizer_model)

Section Name:  Abstract
Generated Summary:   crowd counting problem aims to count the number of objects within an image or a
frame in the videos and is usually solved by estimating the density map
generated from the object location annotations. in contrast to traditional
methods which do not differentiate the density prediction of these two states,
we propose to use a dedicated network branch to predict the object/non-object
mask and then combine its prediction with the input image to produce the density
map. our rationale is that the mask prediction could be better modeled as a
binary segmentation problem and the difficulty of estimating the density could
be reduced if the mask is known. to this end, we study five possible solutions,
and via analysis and experimental validation we identify the most effective one.
through extensive experiments on five public datasets, we demonstrate the
superior performance of the proposed approach over the baselines and show that
our network could ach

# 3. Generate PPT

In [18]:
section_texts = []
section_names = []
section_image_paths = []
if model_name == "gpt-3.5-turbo-0125":
    with open(f"gpt_summaries/{filename}.json", encoding='utf-8') as f:
        data = json.load(f)
    for section in data:
        section_names.append(section["Section"])
        section_texts.append(get_section_groundtruth(section)) # Use GPT generated summaries
        section_image_paths.append(get_section_image_paths(section))
else:
    with open(f"processed/{filename}.json", encoding='utf-8') as f:
        data = json.load(f)
    for section in data:
        section_names.append(section["Section"])
        section_texts.append(get_section_summary(section)) # Use pretrained model generated summaries
        section_image_paths.append(get_section_image_paths(section))

c:\Users\Wang Zihan\.conda\envs\dba5102\lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Create the slide json

In [22]:
if use_gpt_ppt_parsing:
    title_slide_data = get_title_slide_data(filename)
    toc_slide_data = get_toc_slide_data(filename, section_names)
    content_slide_datas = get_content_slide_datas(section_names, section_texts)
else:
    title_slide_data = {'title': filename, 'subtitle': 'Presentation subtitle'}
    toc_slide_data = {'title': 'Table of Contents', 'content': [{'text': filename, 'indent_level': 0}]}
    for section in data:
        if section['Section'] == "No_title":
            continue
        toc_slide_data['content'].append({'text': section['Section'], 'indent_level': 1})
    content_slide_datas = []
    for i, section in enumerate(data):
        if section['Section'] == "No_title":
            continue
        slide = {'title': section['Section'],
                'content': []}
        
        sents = sent_tokenize(section_texts[i])
        for sent in sents[1:]:
            slide['content'].append({'text': sent, 'indent_level': 0})

        content_slide_datas.append([slide])

In [23]:
theme = "Parcel"
prs = generate_section_level_ppt(theme, title_slide_data, toc_slide_data, content_slide_datas, section_image_paths)
prs.save(f"powerpoints/{filename}.pptx")